In [1]:
import geopandas as gpd
import xarray as xr
import pandas as pd

import pint_xarray
import pint_pandas
import pint

from summaflow import (
    GeoLayer,
    SUMMAWorkflow,
)

import os
import glob

In [2]:
# paths
root_path_layers = '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/test-stats/'

# layers' paths
landcover_path = os.path.join(root_path_layers, 'landsat-landcover')
soilclass_path = os.path.join(root_path_layers, 'usda-soil')
merithdyr_path = os.path.join(root_path_layers, 'merit-hydro')

# geolayer's path
root_path_geoms = '../../../examples/bow-at-calgary-geofabric/'
riv_path = os.path.join(root_path_geoms, 'bcalgary_rivers.shp')
cat_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')
hru_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')

# forcings path
root_path_forcings = '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings'

In [3]:
riv_obj = gpd.read_file(riv_path)
cat_obj = gpd.read_file(cat_path)
hru_obj = gpd.read_file(hru_path)

In [4]:
# layers needed by the setup workflow
# elevation
elv = GeoLayer.from_maf(
    maf_stats=os.path.join(merithdyr_path, 'summaflow_stats_elv.csv'),
    maf_layer=os.path.join(merithdyr_path, 'summaflow_elv.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'meters',
)
# landcover
landcover = GeoLayer.from_maf(
    maf_stats=os.path.join(landcover_path, 'summaflow_stats_NA_NALCMS_landcover_2020_30m.csv'),
    maf_layer=os.path.join(landcover_path, 'summaflow_NA_NALCMS_landcover_2020_30m.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)
# USDA soil classes
soil = GeoLayer.from_maf(
    maf_stats=os.path.join(soilclass_path, 'summaflow_stats_soil_classes.csv'),
    maf_layer=os.path.join(soilclass_path, 'summaflow_soil_classes.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)

# dummy layers for `tan_slope`, `contourLength` and `downHRUindex`
# until relevant workflows are implemented inside `gistool`
slope = elv
contour = elv
hru_index = elv

In [5]:
exp = SUMMAWorkflow(
    forcing_data = glob.glob(os.path.join(root_path_forcings, '**', '*.nc'), recursive=True),
    forcing_name_mapping = {
        'pptrate': 'precipitation',
        'airtemp': 'air_temperature',
    },
    forcing_unit_mapping = {
        'precipitation': 'meters/hour',
        'air_temperature': 'degC',
    },
    forcing_to_unit_mapping = {
        'precipitation': 'mm/s',
        'air_temperature': 'K',
    },
    forcing_attrs = {
        'measurement_height': 40,
        'measurement_height_unit': 'meters',
        'forcing_time_zone': 'utc', # original timezone of the forcing datatset
        'target_time_zone': 'America/Edmonton', # if UTC, SUMMA converts to local time zone internally
        'local': {
            'pr': {
                'long_name': 'precipitation at the surface blah blah',
            },
            'temp': {
                'long_name': 'Air temperature',
            },
        },
        'global': {},
    },
    topology_data = {
        'riv': riv_obj,
        'hru': hru_obj,
        'cat': cat_obj,
    },
    topology_unit_mapping = {'a': 'm', 'c': 'ha'},
    topology_to_unit_mapping = {'a': 'bar', 'c': 'dca'},
    topology_attrs = {
        'gru_fid': 'COMID',
        'hru_fid': 'COMID',
    },
    geospatial_data = {
        'elevation': elv,
        'soilTypeIndex': soil,
        'vegTypeIndex': landcover,
        'tan_slope': slope,
        'contourLength': contour,
        'downHRUindex': hru_index,
    },
    settings = {
        'model_path': '/Users/kasrakeshavarz/Desktop/test',
    },
    cold_state = {
        'layers': {
            'nSoil': 8,
            'nSnow': 0,
        },
        'states': {
            'scalarCanopyIce': 0,
            'scalarCanopyLiq': 0,
            'scalarSnowDepth': 0,
            'scalarSWE': 0,
            'scalarSfcMeltPond': 0,
            'scalarAquiferStorage': 0.4,
            'scalarSnowAlbedo': 0,
            'scalarCanairTemp': 283.16,
            'scalarCanopyTemp': 283.16,
            'mLayerTemp': 283.16,
            'mLayerVolFracIce': 0,
            'mLayerVolFracLiq': 0.4,
            'mLayerMatricHead': -1.0,
            'mLayerDepth': [0.025, 0.075, 0.15, 0.25, 0.5, 0.5, 1, 1.5],
        },
    },
    auxillary = {
        # 'dt_init': 450 # if not provided, defaults to forcing data timestep
    }
)

/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/src/summaflow/core.py:257: UserWarning: `dt_init` not provided in auxillary dictionary; defaulting to forcing time-step.
  warnings.warn("`dt_init` not provided in auxillary dictionary;"


In [6]:
exp

Forcing files: ['/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201101.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201102.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201103.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201104.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201105.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201106.nc', '/User

## Workflow tests

In [7]:
exp.init_attrs(return_ds=False)

/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/src/summaflow/utils.py:133: UserWarning: Using EPSG:3857 for centroid calculations on geographic data. For better accuracy, specify a local projection using calculation_crs parameter.
  warnings.warn(


In [8]:
exp.init_forcing(return_ds=False)

In [9]:
exp.init_cold_state(return_ds=False)

In [10]:
exp

Forcing files: ['/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201101.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201102.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201103.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201104.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201105.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201106.nc', '/User